In [ ]:
import pandas as pd
import numpy as np
import torch
import sys

from neural_models import NeuralLVModel, NeuralLSVModel, NeuralTrainer

In [ ]:
rate = 0.025
device = 'cuda:0'

model = NeuralLSVModel(input_dim=1, # NeuralLSVModel
                       hidden_dims=[50,50,50,50],
                       rate=rate, 
                       device=device)
problem = 'lower bound' # 'lower bound', 'standard' or 'upper bound'

trainer = NeuralTrainer(model, problem)

data = pd.read_csv('Call_Put_prices.csv', index_col=0)
time_grid = torch.linspace(0, 1, 96+1)
checkpoints = [3e-5, 1e-5, 
               5e-6, 3e-6, 2e-6, 1e-6, 
               7e-7, 5e-7, 4e-7, 3e-7, 2e-7, 1e-7, 
               8e-8, 7e-8, 6e-8, 5e-8, 4e-8, 3e-8, 2e-8, 1e-8]
schedule = 'steps'

trainer.prepare_training(data, time_grid, device, schedule)

In [ ]:
with open(f"logs_{model.__name__}_{problem.replace(' ', '')}.txt", 'w') as log_file:
    try:
        sys.stdout = log_file
        training_records = trainer.train(s0=1., 
                                         N_trn=40000, 
                                         N_batchs=20, 
                                         N_epochs=1000, 
                                         checkpoints=checkpoints,
                                         verbose=True)
        trainer.plot_metrics()
    except KeyboardInterrupt:
        trainer.save()
        print("\nTraining interrupted by user.")
    finally:
        # Restore stdout to its original state
        sys.stdout = sys.__stdout__
        # Close the file
        log_file.close()